In [33]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
import numpy as np

housnig = pd.read_csv('./housing.csv', header=0, sep=',')

train_set, test_set = train_test_split(housnig, test_size=0.2, random_state=42)
# train_set.shape
train_set.head()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,103000.0,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,382100.0,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,172600.0,NEAR OCEAN
14265,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,93400.0,NEAR OCEAN
2271,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,96500.0,INLAND


In [34]:
# ============== Prepare the Data ====================
# ======== Number Data --> missing value
# ======== categorical and text data ---> labelEncoder, oneHotEncoder
# ======== Numerical data ===> Feature scaling
# ======== Numerical date ===> Custom transformers

df = train_set.copy()

# چون میخوایم روی این قسمت از داده ماشین رو اموزش بدیم پس خودش رو باید حذف کنیم
df_label = df['median_house_value'].copy()
df = df.drop('median_house_value', axis=1)
df.info()

# ocean_proximity value is nan because remove this from data-set
df_num = df.drop('ocean_proximity', axis=1)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 14196 to 15795
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16508 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   ocean_proximity     16512 non-null  object 
dtypes: float64(8), object(1)
memory usage: 1.3+ MB


In [35]:
from sklearn.preprocessing import StandardScaler


class FitAndTransform():
    def fit(self, input, strategy="median"):
        # fix all null row value by stragegy
        imputer = SimpleImputer(missing_values=np.nan, strategy=strategy)
        # fit get all data and learn for transform
        imputer.fit(input)
        input_impute_tr = self.transform(input, imputer)
        return input_impute_tr

    def transform(self, input, imputerValue):
        # transform for change face
        # X is np arra should change to pandas
        X = imputerValue.transform(input)
        input_impute_tr = pd.DataFrame(X, columns=input.columns)
        return input_impute_tr


def columnsAppender(mainData, columnsNameList):
    columns = list(mainData.columns)
    for columnName in columnsNameList:
        columns.append(columnName)
    return columns


def combinedAttributesAddr(inputValue, rooms_ix, bedrooms_ix, population_ix, household_ix):
    rooms_per_households = inputValue[:,
                                      rooms_ix] / inputValue[:, household_ix]
    population_per_household = inputValue[:,
                                          population_ix] / inputValue[:, household_ix]
    bedrooms_per_rooms = inputValue[:, bedrooms_ix] / inputValue[:, rooms_ix]
    data_custom_tr_tmp = np.c_[
        inputValue, rooms_per_households, population_per_household, bedrooms_per_rooms]
    return pd.DataFrame(data_custom_tr_tmp)


# =============== feature scaling ================
# Standardization => range of -n,+n      Normalization => [0, 1]
# وقتی اختلاف مین و مکس حیلی زیاد باشه. مثلا از 2 به 3900 الگوریتم های ما نمیتونه درست نصمیم بگیره.
# برای همین اسکیل میکنیم تا دز مقیاس منطقی تری الگوریتم رو در بیاریم.

# make standardScaler generator

def numStandardScaler(inputData):
    feature_scal = StandardScaler()
    data_num_scaled_tr = pd.DataFrame(feature_scal.fit_transform(
        inputData.values), columns=inputData.columns)
    return data_num_scaled_tr


In [36]:
# test all function

rooms_ix, bedrooms_ix, population_ixrooms_ix, household_ixrooms_ix = 3, 4, 5, 6
custom_columns_listrooms_ix = ['rooms_per_households',
                       'population_per_household', 'bedrooms_per_rooms']

fitAndTransformrooms_ix = FitAndTransform()
data_custom_impure_trrooms_ix = fitAndTransformrooms_ix.fit(df_num)
data_custom_impure_trrooms_ix.info()

data_custom_trrooms_ix = combinedAttributesAddr(
    data_custom_impure_trrooms_ix.values, rooms_ix, bedrooms_ix, population_ixrooms_ix, household_ixrooms_ix)
data_custom_trrooms_ix.columns = columnsAppender(
    data_custom_impure_trrooms_ix, custom_columns_listrooms_ix)

data_num_standard_scaler = numStandardScaler(data_custom_trrooms_ix)
data_num_standard_scaler.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16512 entries, 0 to 16511
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16512 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
dtypes: float64(8)
memory usage: 1.0 MB


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_households,population_per_household,bedrooms_per_rooms
0,1.272587,-1.372811,0.348490,0.222569,0.211261,0.768276,0.322906,-0.326196,-0.174916,0.051376,-0.212034
1,0.709162,-0.876696,1.618118,0.340293,0.593164,-0.098901,0.672027,-0.035843,-0.402835,-0.117362,0.341808
2,-0.447603,-0.460146,-1.952710,-0.342597,-0.495259,-0.449818,-0.430461,0.144701,0.088216,-0.032280,-0.661803
3,1.232698,-1.382172,0.586545,-0.561490,-0.409331,-0.007434,-0.380587,-1.017864,-0.600015,0.077507,0.782552
4,-0.108551,0.532084,1.142008,-0.119565,-0.256570,-0.485877,-0.314962,-0.171488,0.349007,-0.068832,-0.550535


In [40]:
# option -> OneHotEncoder -> add any  columns like checkList => 0,1
from sklearn.preprocessing import OneHotEncoder

def oneHotEncoding(colName):
    # sparse = false for null number
    encoder_1hot = OneHotEncoder(sparse=False)
    data_cat_1hot_tmp = encoder_1hot.fit_transform(df[[colName]])
    data_cat_1hot = pd.DataFrame(data_cat_1hot_tmp)
    data_cat_1hot.columns = encoder_1hot.get_feature_names_out(
        input_features=[colName])
    return data_cat_1hot


data_cat_1hot.head()








,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0


In [41]:
final = pd.concat([data_num_standard_scaler, data_cat_1hot], axis=1)
final.head(10)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_households,population_per_household,bedrooms_per_rooms,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,1.272587,-1.372811,0.348490,0.222569,0.211261,0.768276,0.322906,-0.326196,-0.174916,0.051376,-0.212034,0.0,0.0,0.0,0.0,1.0
1,0.709162,-0.876696,1.618118,0.340293,0.593164,-0.098901,0.672027,-0.035843,-0.402835,-0.117362,0.341808,0.0,0.0,0.0,0.0,1.0
2,-0.447603,-0.460146,-1.952710,-0.342597,-0.495259,-0.449818,-0.430461,0.144701,0.088216,-0.032280,-0.661803,0.0,0.0,0.0,0.0,1.0
3,1.232698,-1.382172,0.586545,-0.561490,-0.409331,-0.007434,-0.380587,-1.017864,-0.600015,0.077507,0.782552,0.0,0.0,0.0,0.0,1.0
4,-0.108551,0.532084,1.142008,-0.119565,-0.256570,-0.485877,-0.314962,-0.171488,0.349007,-0.068832,-0.550535,0.0,1.0,0.0,0.0,0.0
5,-1.135679,0.831625,-0.683082,1.099060,0.643289,1.115675,0.790151,1.439919,0.354757,0.023111,-0.901665,1.0,0.0,0.0,0.0,0.0
6,0.803897,-0.750327,-0.048268,-0.439627,-0.289986,-0.062842,-0.293963,-0.715013,-0.456530,0.034142,0.593991,1.0,0.0,0.0,0.0,0.0
7,-1.469745,1.060961,0.665897,-0.054266,-0.335337,-0.375941,-0.217838,2.157529,0.258670,-0.060568,-0.951420,0.0,0.0,0.0,1.0,0.0
8,0.838800,-0.708204,-1.873359,-0.697148,-0.531062,-0.534249,-0.496085,-1.250241,-0.760128,-0.040033,1.168206,1.0,0.0,0.0,0.0,0.0
9,0.893646,-0.759688,-1.873359,7.379811,5.545968,7.036405,5.977096,1.336938,0.542474,0.025712,-1.029966,1.0,0.0,0.0,0.0,0.0


In [39]:
# pipeLine 1:00:00